# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey

# Access maps with unique API key
gmaps.configure(api_key=gkey)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
output_weather_file = "../WeatherPy/output_data/city_weather.csv"
weather_data = pd.read_csv(output_weather_file)

In [3]:
weather_data.head()

,city,country,lat,long,temp,max_temp,humidity,wind speed,clouds
0,Jamestown,AU,-33.21,138.60,40.32,40.32,91,4.47,0
1,Provideniya,RU,64.42,-173.23,34.02,34.02,98,11.45,100
2,Hithadhoo,MV,-0.60,73.08,82.62,82.62,71,11.93,39
3,Meulaboh,ID,4.14,96.13,79.92,79.92,80,6.26,100
4,Kapaa,US,22.08,-159.32,74.50,77.00,83,6.93,75


In [4]:
len(weather_data)

557

In [5]:
# Store latitude and longitude in locations
locations = weather_data[["lat", "long"]]

# Fill NaN values and convert to float
humidity = weather_data["humidity"].astype(float)

In [6]:
len(locations)

557

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
# Customize the size of the figure
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)
fig.add_layer(heat_layer)
fig



Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
# Narrow down the DataFrame to find your ideal weather condition. 
# Drop a row by condition  
# A max temperature >75F and <=80F lower than 80 degrees but higher than 75.
# very few clouds (<5%) cloudiness.
# Wind speed < 10 (light breeze)


myCities = weather_data.loc[weather_data["temp"]>75]
myCities = myCities.loc[myCities["temp"]<80]
myCities = myCities.loc[myCities["wind speed"]<10]
myCities = myCities.loc[myCities["clouds"]<5]



In [9]:
#myCities.count()
myCities

,city,country,lat,long,temp,max_temp,humidity,wind speed,clouds
275,Urubicha,BO,-15.63,-63.08,76.86,76.86,63,2.59,4
551,Karratha,AU,-20.74,116.85,77.01,77.01,32,8.37,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
hotel_df = myCities
hotel_df

,city,country,lat,long,temp,max_temp,humidity,wind speed,clouds
275,Urubicha,BO,-15.63,-63.08,76.86,76.86,63,2.59,4
551,Karratha,AU,-20.74,116.85,77.01,77.01,32,8.37,0


In [11]:
hotel_df["hotel"]=" "

In [12]:
hotel_df["city"]

275    Urubicha
551    Karratha
Name: city, dtype: object

In [13]:
hotel_df.reset_index()

,index,city,country,lat,long,temp,max_temp,humidity,wind speed,clouds,hotel
0,275,Urubicha,BO,-15.63,-63.08,76.86,76.86,63,2.59,4,
1,551,Karratha,AU,-20.74,116.85,77.01,77.01,32,8.37,0,


In [15]:
# find the closest hotel of each type to coordinates
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
radius = 5000
params = {
    "location": "",  # philadelphia coords,
    "rankby": "distance",
    "keyword": "hotel",
    "type": "hotel",
    "key": gkey,
}
# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    params["location"] = f"{hotel_df.loc[index, 'lat']},{hotel_df.loc[index, 'long']}"

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()

        
    # extract results
    results = response['results']
    


    try:
        hotel_df.loc[index, 'hotel'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

------------
------------


In [16]:
hotel_df

,city,country,lat,long,temp,max_temp,humidity,wind speed,clouds,hotel
275,Urubicha,BO,-15.63,-63.08,76.86,76.86,63,2.59,4,HOTEL IGUASU
551,Karratha,AU,-20.74,116.85,77.01,77.01,32,8.37,0,ibis Styles Karratha


In [17]:
hotel_df=hotel_df[hotel_df["hotel"]!=" "]
hotel_df

,city,country,lat,long,temp,max_temp,humidity,wind speed,clouds,hotel
275,Urubicha,BO,-15.63,-63.08,76.86,76.86,63,2.59,4,HOTEL IGUASU
551,Karratha,AU,-20.74,116.85,77.01,77.01,32,8.37,0,ibis Styles Karratha


In [18]:
narrowed_city_df = hotel_df[["hotel","city","country","lat","long"]]
narrowed_city_df=narrowed_city_df.rename(columns={"hotel": "Hotel Name","city": "City","country":"Country","lat":"Lat","long":"Lng"})
narrowed_city_df.head()

,Hotel Name,City,Country,Lat,Lng
275,HOTEL IGUASU,Urubicha,BO,-15.63,-63.08
551,ibis Styles Karratha,Karratha,AU,-20.74,116.85


In [19]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = narrowed_city_df[["Lat", "Lng"]]

In [20]:
len(narrowed_city_df)

2

In [21]:
# Add marker layer ontop of heat map

# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations,info_box_content=hotel_info)


# Add the layer to the map
fig.add_layer(markers)
fig
# Display Map

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…